# Properties of the reconciled distribution via conditioning

# Introduction

This vignette reproduces the results of the paper *Properties of the reconciled distributions for Gaussian and count forecasts* (Zambon et al. 2024), accepted for publication in the International Journal of Forecasting. We replicate here the results obtained in the original [R package](https://cran.r-project.org/web/packages/bayesRecon/vignettes/reconciliation_properties.html).

# Data and base forecasts

The R package released a new data set, containing time series of counts of extreme market events in five economic sectors in the period 2005-2018 (3508 trading days). The counts are computed by considering 29 companies included in the Euro Stoxx 50 index and observing if the value of the CDS spread on a given day exceeds the 90-th percentile of its distribution in the last trading year. The companies are divided into the following sectors: Financial (FIN), Information and Communication Technology (ICT), Manufacturing (MFG), Energy (ENG), and Trade (TRD).

The hierarchy is composed of 5 bottom time series, the daily number of extreme market events in each sector, and 1 upper time series (the sum of the different sectors). Data are stored in `extr_mkt_events.pkl`.
![extr_mkt_events.jpeg](../pictures/extr_mkt_events.jpeg)

The python counterpart of the base forecasts are stored in `extr_mkt_events_basefc.pkl` which are to be reconciled. They are produced using the model by (Agosto 2022); the predictive distributions are negative binomial.

In [5]:
import numpy as np
import pandas as pd
import time
from scipy.stats import nbinom
from numpy.random import default_rng
from bayesreconpy.reconc_BUIS import reconc_BUIS

n_b = 5
n_u = 1
n = n_b + n_u

A = np.ones((n_u, n_b))

# Actual values:
actuals = pd.read_pickle('../data/extr_mkt_events.pkl')
#Bse forecasts:
base_fc = (pd.read_pickle('../data/extr_mkt_events_basefc.pkl'))

N = actuals.shape[0]  # number of days (3508)

"""
If you want to run only N reconciliations (instead of 3508)
N = 200
actuals = actuals[:N,:]
base_fc['mu'] = np.array(base_fc['mu'])[:N,:]
"""

"\nIf you want to run only N reconciliations (instead of 3508)\nN = 200\nactuals = actuals[:N,:]\nbase_fc['mu'] = np.array(base_fc['mu'])[:N,:]\n"

# Reconciliation via conditioning

We reconcile the base forecasts via conditioning, using importance sampling. We use the `reconc_BUIS` function, which implements the BUIS algorithm (Zambon, Azzimonti, and Corani 2024); since there is only one upper time series in the hierarchy, the BUIS algorithm is equivalent to importance sampling. We perform 3508 reconciliations, one for each day, drawing each time 10,000 samples from the reconciled distribution. We use 10,000 samples instead of 100,000 (as in the paper) to speed up the computation.

For each day, we save the empirical mean, median, and quantiles of the reconciled distribution.

In [6]:
# Initialize matrices to store mean, median, lower, and upper quantiles
rec_means = np.full((N, n), np.nan)
rec_medians = np.full((N, n), np.nan)
rec_L = np.full((N, n), np.nan)
rec_U = np.full((N, n), np.nan)

# Interval coverage
int_cov = 0.9
q1 = (1 - int_cov) / 2
q2 = (1 + int_cov) / 2

# Number of samples for reconciled distribution
N_samples = int(1e4)
rng = default_rng(42)  # Seeded random number generator for reproducibility

# Start timing
start = time.time()

for j in range(N):
    # Prepare base forecasts for the current iteration
    base_fc_j = []
    for i in range(n):
        # Fetch `size` and `mu` for each forecast from the corresponding DataFrame column and row
        size_value = base_fc['size'].iloc[0, i]  # Use row 0 of `size` for all j's
        mu_value = base_fc['mu'].iloc[j, i]
        base_fc_j.append({"size": size_value, "mu": mu_value})

    # Reconcile via importance sampling
    buis = reconc_BUIS(A, base_fc_j, "params", "nbinom", num_samples=N_samples, seed=42)
    samples_y = buis['reconciled_samples']

    # Save mean, median, and quantiles
    rec_means[j, :] = np.mean(samples_y, axis=1)  # Mean along rows
    rec_medians[j, :] = np.median(samples_y, axis=1)  # Median along rows
    rec_L[j, :] = np.quantile(samples_y, q1, axis=1)  # Lower quantile
    rec_U[j, :] = np.quantile(samples_y, q2, axis=1)  # Upper quantile

# End timing
stop = time.time()

# Print computation time
print(f"Computational time for {N} reconciliations: {round(stop - start, 2)} seconds")

Computational time for 3508 reconciliations: 20.13 seconds


We compute the median and the quantiles of the negative binomial base forecasts.

In [7]:
# Initialize matrices
base_means = base_fc['mu'].values  # Convert to numpy array for easier manipulation
base_medians = np.full((N, n), np.nan)
base_L = np.full((N, n), np.nan)
base_U = np.full((N, n), np.nan)

# Loop through each column (i.e., each forecast variable)
for i in range(n):
    size_value = base_fc['size'].iloc[0, i]  # Use row 0 of `size` for each i (if size is constant across rows)

    # Calculate the median, lower, and upper quantiles for each value of `mu` in the column
    base_medians[:, i] = [nbinom.ppf(0.5, size_value, 1 / (1 + mu)) for mu in base_means[:, i]]
    base_L[:, i] = [nbinom.ppf(q1, size_value, 1 / (1 + mu)) for mu in base_means[:, i]]
    base_U[:, i] = [nbinom.ppf(q2, size_value, 1 / (1 + mu)) for mu in base_means[:, i]]

For each day and for each time series, we compute the absolute error, the squared error, and the interval score for the base and reconciled forecasts.

In [8]:
# Compute the squared errors
SE_base = (base_means - actuals) ** 2
SE_rec = (rec_means - actuals) ** 2

# Compute the absolute errors
AE_base = np.abs(base_medians - actuals)
AE_rec = np.abs(rec_medians - actuals)

# Define the interval score function
def interval_score(l, u, actual, int_cov=0.9):
    is_score = (u - l) + \
               (2 / (1 - int_cov)) * (actual - u) * (actual > u) + \
               (2 / (1 - int_cov)) * (l - actual) * (l > actual)
    return is_score

# Vectorized computation of interval scores
IS_base = np.vectorize(interval_score)(base_L, base_U, actuals)
IS_rec = np.vectorize(interval_score)(rec_L, rec_U, actuals)

# Reshape the results into N x n matrices if needed
IS_base = IS_base.reshape(N, n)
IS_rec = IS_rec.reshape(N, n)

We compute and show the skill scores, which measure the improvement of the reconciled forecasts over the base forecasts. The skill score is symmetric and bounded between -2 and 2.

In [9]:
# Compute skill scores (SS) for Absolute Error, Squared Error, and Interval Score
SS_AE = (AE_base - AE_rec) / (AE_base + AE_rec) * 2
SS_SE = (SE_base - SE_rec) / (SE_base + SE_rec) * 2
SS_IS = (IS_base - IS_rec) / (IS_base + IS_rec) * 2

# Replace NaN values with 0
SS_AE = np.nan_to_num(SS_AE, nan=0)
SS_SE = np.nan_to_num(SS_SE, nan=0)
SS_IS = np.nan_to_num(SS_IS, nan=0)

# Calculate column means for each skill score matrix
mean_skill_scores = np.round([
    SS_IS.mean(axis=0),
    SS_SE.mean(axis=0),
    SS_AE.mean(axis=0)
], 2)

# Convert to DataFrame and structure the output
mean_skill_scores_df = pd.DataFrame(mean_skill_scores,
                                    index=["Interval score", "Squared error", "Absolute error"],
                                    columns=actuals.columns if isinstance(actuals, pd.DataFrame) else range(n))

# Display the results as a table
print(mean_skill_scores_df.to_markdown())

|                |   ALL |   FIN |   ICT |   MFG |   ENG |   TRD |
|:---------------|------:|------:|------:|------:|------:|------:|
| Interval score |  1.36 |  1.41 |  1.67 |  1.95 |  1.72 |  1.75 |
| Squared error  |  0.83 |  1.12 |  1.13 |  1.07 |  1.12 |  1.09 |
| Absolute error |  0.73 |  0.25 |  0.49 |  1.96 |  0.5  |  0.61 |


The table closely matches Table 4 of the paper. In order to exactly reproduce the paper table, it is necessary increase the number of samples drawn from the reconciled distribution to 100,000.

# Reconciled mean and variance

We now show the effects of the reconciliation on the mean and variance of the forecast distribution. For more details, we refer to Section 3.2 of the paper.

We observe two different behaviors for the reconciled upper mean: it can be between the base and the bottom-up mean (*combination* effect) or it can be lower than both (*concordant-shift* effect). We show them for two different days.

In [10]:
# Define the number of samples to draw
N_samples = int(1e5)

# Example of concordant-shift effect for j = 123 (124 in R)
j = 123
base_fc_j = []

# Prepare base forecast for the specified index
for i in range(n):
    size_value = base_fc['size'].iloc[0, i]  # size is constant across rows
    mu_value = base_fc['mu'].iloc[j, i]
    base_fc_j.append({"size": size_value, "mu": mu_value})

# Reconcile via importance sampling
buis = reconc_BUIS(A, base_fc_j, "params", "nbinom", num_samples=N_samples, seed=42)
samples_y = buis['reconciled_samples']

# Compute the means
base_upper_mean = round(base_fc['mu'].iloc[j, 0], 2)
bottom_up_upper_mean = round(base_fc['mu'].iloc[j, 1:].sum(), 2)
reconciled_upper_mean = round(np.mean(samples_y[0, :]), 2)

# Display results in a structured format
means = [base_upper_mean, bottom_up_upper_mean, reconciled_upper_mean]
col_names = ["Base upper mean", "Bottom-up upper mean", "Reconciled upper mean"]

# Create a DataFrame to display the results as a table
means_df = pd.DataFrame([means], columns=col_names)
print(means_df.to_markdown())

|    |   Base upper mean |   Bottom-up upper mean |   Reconciled upper mean |
|---:|------------------:|-----------------------:|------------------------:|
|  0 |             10.67 |                    9.1 |                    8.29 |


In [11]:
j = 1699
base_fc_j = []

# Prepare base forecast for the specified index
for i in range(n):
    size_value = base_fc['size'].iloc[0, i]  # Assume size is constant across rows
    mu_value = base_fc['mu'].iloc[j, i]
    base_fc_j.append({"size": size_value, "mu": mu_value})

# Reconcile via importance sampling
buis = reconc_BUIS(A, base_fc_j, "params", "nbinom", num_samples=N_samples, seed=42)
samples_y = buis['reconciled_samples']

# Compute the means
base_upper_mean = round(base_fc['mu'].iloc[j, 0], 2)
bottom_up_upper_mean = round(base_fc['mu'].iloc[j, 1:].sum(), 2)
reconciled_upper_mean = round(np.mean(samples_y[0, :]), 2)

# Display results in a structured format
means = [base_upper_mean, bottom_up_upper_mean, reconciled_upper_mean]
col_names = ["Base upper mean", "Bottom-up upper mean", "Reconciled upper mean"]

# Create a DataFrame to display the results as a table
means_df = pd.DataFrame([means], columns=col_names)
print(means_df.to_markdown())

|    |   Base upper mean |   Bottom-up upper mean |   Reconciled upper mean |
|---:|------------------:|-----------------------:|------------------------:|
|  0 |             26.51 |                  43.39 |                   35.82 |


Finally, we show an example in which the variance of the bottom time series increases after reconciliation. This is a major difference with the Gaussian reconciliation, for which the reconciled variance is always smaller than the base variance.

In [12]:
j = 2307  

# Prepare base forecast for the specified index
base_fc_j = []
for i in range(n):
    size_value = base_fc['size'].iloc[0, i]  # Assume size is constant across rows
    mu_value = base_fc['mu'].iloc[j, i]
    base_fc_j.append({"size": size_value, "mu": mu_value})

# Reconcile via importance sampling
buis = reconc_BUIS(A, base_fc_j, "params", "nbinom", num_samples=N_samples, seed=42)
samples_y = buis['reconciled_samples']

# Compute variance of the base bottom forecasts
base_bottom_var = [
    np.var(np.random.negative_binomial(n=size, p=size / (size + mu), size=int(1e5)))
    for mu, size in zip(base_fc['mu'].iloc[j, 1:], 
                        base_fc['size'].iloc[0, 1:])
]

# Compute variance of the reconciled bottom forecasts
rec_bottom_var = np.var(samples_y[1:, :], axis=1)

# Combine base and reconciled variances and display results
bottom_var = np.vstack([base_bottom_var, rec_bottom_var])
bottom_var_df = pd.DataFrame(bottom_var, index=["var base", "var reconc"])

# Display as a table with two decimal places
print(bottom_var_df.round(2).to_markdown())

|            |     0 |    1 |    2 |    3 |    4 |
|:-----------|------:|-----:|-----:|-----:|-----:|
| var base   | 11.43 | 2.23 | 1.48 | 0.34 | 1.38 |
| var reconc | 14.05 | 2.57 | 1.64 | 0.38 | 1.53 |


The results here match exactly with the ones available in the [original R vignette](https://cran.r-project.org/web/packages/bayesRecon/vignettes/reconciliation_properties.html)